In [1]:
import torch
import numpy as np
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, precision_recall_curve
from torch.nn.utils import clip_grad_norm_
import matplotlib.pyplot as plt
from torch_geometric.nn import BatchNorm
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
from scipy.stats import skew, kurtosis, linregress
from scipy.signal import find_peaks
import einops

import lovely_tensors as lt
lt.monkey_patch()

/home/pasquale/miniconda3/envs/gnnleak/lib/python3.12/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/pasquale/miniconda3/envs/gnnleak/lib/python3.12/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "


In [2]:
#%% Data Loading and Preprocessing
print("Loading and preprocessing data...")
graph_data = torch.load('data/graph_data_torch.pt')

Loading and preprocessing data...


/tmp/ipykernel_906759/1167354728.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_data = torch.load('data/graph_data_torch.pt')


In [3]:
num_nodes = graph_data['node_features'].shape[1]
num_edges = graph_data['edge_index'].shape[0]

In [4]:
graph_data

{'scenario': tensor[17520000, 1] i32 67Mb x∈[1, 1000] μ=500.500 σ=288.675,
 'node_features': tensor[17520000, 32] n=560640000 (2.1Gb) x∈[0., 69.985] μ=61.383 σ=11.861,
 'edge_attr': tensor[17520000, 34] n=595680000 (2.2Gb) x∈[-3.726e+03, 1.344e+04] μ=835.518 σ=1.509e+03,
 'edge_index': tensor[34, 2] i64 n=68 x∈[0, 31] μ=15.471 σ=9.229,
 'y': tensor[17520000, 32] i64 n=560640000 (4.2Gb) x∈[0, 1] μ=0.008 σ=0.087}

In [5]:
graph_data["edge_attr"] = torch.cat([graph_data["edge_attr"], graph_data["edge_attr"]], dim=1)
edges_reversed = torch.stack([reversed(e) for e in graph_data["edge_index"].reshape(-1, 2)])
edge_index = torch.cat([graph_data["edge_index"], edges_reversed], dim=0)
graph_data["edge_index"] = edge_index

### Feature Extraction

In [28]:
# Temporal parameters
WINDOW_SIZE = 24     
SUBSAMPLE_FACTOR = 10  # Reduced subsampling for more windows
STRIDE = 12
num_nodes = graph_data['node_features'].shape[1]

def preprocess_data(features, window_size, stride, subsample):
    """Process features into [windows, nodes] format with mean, max, min, and std aggregation"""
    # Slice data into overlapping windows
    features = features.unfold(0, window_size, stride).float()
    
    # Compute mean, max, min, and std pooling within each window
    mean_values = features.mean(dim=2)  # Mean pooling
    max_values = features.max(dim=2).values  # Max pooling
    min_values = features.min(dim=2).values  # Min pooling
    std_values = features.std(dim=2)  # Standard deviation pooling
    
    # Combine mean, max, min, and std into a single representation
    combined_features = torch.cat((mean_values, max_values, min_values, std_values), dim=1)  # Concatenate along feature axis
    
    # Subsample the windows
    return combined_features[::subsample, :]

In [29]:
# Process features and labels
node_features = preprocess_data(graph_data['node_features'], WINDOW_SIZE, SUBSAMPLE_FACTOR)
edge_features = preprocess_data(graph_data['edge_attr'], WINDOW_SIZE, SUBSAMPLE_FACTOR)
window_labels = graph_data['y'].unfold(0, WINDOW_SIZE, WINDOW_SIZE).max(dim=2).values[::SUBSAMPLE_FACTOR, :]
window_scenarios = graph_data['scenario'].unfold(0, WINDOW_SIZE, WINDOW_SIZE).mode(dim=2).values[::SUBSAMPLE_FACTOR, :]
print(node_features)
print(edge_features)
print(window_labels)
print(window_scenarios)

tensor[146000, 128] n=18688000 (71Mb) x∈[0., 69.984] μ=46.181 σ=27.628
tensor[146000, 272] n=39712000 (0.1Gb) x∈[-3.676e+03, 1.337e+04] μ=687.584 σ=1.380e+03
tensor[146000, 32] i64 n=4672000 (36Mb) x∈[0, 1] μ=0.008 σ=0.088
tensor[146000, 1] i32 0.6Mb x∈[1, 1000] μ=500.502 σ=288.676


In [13]:
torch.save({
    'node_features': node_features,
    'window_labels': window_labels,
    "edge_features": edge_features,
    'window_scenarios': window_scenarios,
    "edge_index": graph_data['edge_index'].long().t().contiguous(),
},
    "data/processed_data_W12_S10.pt"           
)

## More Features

In [6]:
from scipy.fft import fft
from scipy.stats import entropy
import statsmodels.api as sm

In [49]:
# Temporal parameters
WINDOW_SIZE = 48
STRIDE = 24
SUBSAMPLE_FACTOR = 1  # Reduced subsampling for more windows

def permutation_entropy(time_series, m=3, tau=1):
    """Compute permutation entropy for a time series."""
    results = []
    for series in time_series:
        permutations = []
        n = len(series)

        for i in range(n - (m - 1) * tau):
            vec = series[i:i + m * tau:tau]
            permutation = np.argsort(vec)
            permutations.append(tuple(permutation))

        unique_perms, counts = np.unique(permutations, axis=0, return_counts=True)
        probs = counts / len(permutations)
        results.append(entropy(probs, base=2))
    results = np.array(results)
    return results

def fourier_entropy(time_series):
    """Compute Fourier entropy for a time series."""
    fft_coeffs = fft(time_series)
    power_spectrum = np.abs(fft_coeffs) ** 2
    normalized_spectrum = power_spectrum / np.sum(power_spectrum)
    return entropy(normalized_spectrum, base=2, axis=-1)

def partial_autocorrelation(time_series, lag):
    """Compute partial autocorrelation for a time series at a given lag."""
    if time_series.ndim == 2:
        # If input is 2D (batch of time series), compute PACF for each time series
        pacf_values = np.array([sm.tsa.stattools.pacf(ts, nlags=lag)[lag] for ts in time_series])
        return pacf_values
    elif time_series.ndim == 1:
        # If input is 1D (single time series), compute PACF directly
        return sm.tsa.stattools.pacf(time_series, nlags=lag)[lag]
    else:
        raise ValueError("Input must be 1D or 2D.")

def preprocess_data(features, window_size, stride, subsample):
    """Process features into [windows, nodes] format."""
    # Slice data into overlapping windows
    features = features.unfold(0, window_size, stride).float()
    
    # Compute mean, max, min, std for each window
    mean_values = features.mean(dim=2)  # Mean pooling
    max_values = features.max(dim=2).values  # Max pooling
    min_values = features.min(dim=2).values  # Min pooling
    std_values = features.std(dim=2)  # Standard deviation pooling
    
    # Compute additional features for each window
    perm_entropy_values = torch.tensor([permutation_entropy(window) for window in tqdm(features.numpy(), desc="permutation entropy")])
    fourier_entropy_values = torch.tensor([fourier_entropy(window) for window in tqdm(features.numpy(), desc="fourier entropy")])
    pacf_values = torch.tensor([partial_autocorrelation(window, lag=0) for window in tqdm(features.numpy(), desc="partial autocorrelation entropy")])
    
    # Stack all features together
    features = torch.stack((mean_values, max_values, min_values, std_values, 
                            perm_entropy_values, fourier_entropy_values, pacf_values), dim=1)
    
    # Rearrange features into [windows, nodes] format
    features = einops.rearrange(features, "l f n -> l (f n)")
    
    # Subsample the windows
    return features[::subsample, :]

In [62]:
# Process features and labels
node_features = graph_data['node_features']
edge_features = graph_data['edge_attr']
print("Extracting Node Fetures")
node_features = preprocess_data(node_features, WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
print("Extracting Edge Fetures")
# edge_features = einops.rearrange(graph_data['edge_attr'], "l e -> (l e)")
edge_features = preprocess_data(graph_data['edge_attr'], WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
window_labels = graph_data['y'].unfold(0, WINDOW_SIZE, STRIDE).max(dim=2).values[::SUBSAMPLE_FACTOR, :]
window_scenarios = graph_data['scenario'].unfold(0, WINDOW_SIZE, STRIDE).mode(dim=2).values[::SUBSAMPLE_FACTOR, :]
print(node_features)
print(edge_features)
print(window_labels)
print(window_scenarios)

Extracting Node Fetures


permutation entropy:  55%|█████▌    | 401802/729999 [1:11:13<58:11, 94.01it/s]  


KeyboardInterrupt: 

## Windowing

In [6]:
# Temporal parameters
WINDOW_SIZE = 24
SUBSAMPLE_FACTOR = 1  # Reduced subsampling for more windows
STRIDE = 18
num_nodes = graph_data['node_features'].shape[1]

def preprocess_data(features, window_size, stride, subsample):
    """Process features into [windows, nodes] formatn"""
    # Slice data into overlapping windows
    features = features.unfold(0, window_size, stride).float()
    features = einops.rearrange(features, "l n w -> l (w n)")
    
    # Subsample the windows
    return features[::subsample, :]

In [7]:
# Process features and labels
node_features = preprocess_data(graph_data['node_features'], WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
edge_features = preprocess_data(graph_data['edge_attr'], WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
window_labels = graph_data['y'].unfold(0, WINDOW_SIZE, STRIDE).max(dim=2).values[::SUBSAMPLE_FACTOR, :]
window_scenarios = graph_data['scenario'].unfold(0, WINDOW_SIZE, STRIDE).mode(dim=2).values[::SUBSAMPLE_FACTOR, :]
print(node_features)
print(edge_features)
print(window_labels)
print(window_scenarios)

tensor[973333, 768] n=747519744 (2.8Gb) x∈[0., 69.985] μ=61.383 σ=11.861
tensor[973333, 1632] n=1588479456 (5.9Gb) x∈[-3.726e+03, 1.344e+04] μ=835.521 σ=1.509e+03
tensor[973333, 32] i64 n=31146656 (0.2Gb) x∈[0, 1] μ=0.008 σ=0.088
tensor[973333, 1] i32 3.7Mb x∈[1, 1000] μ=500.500 σ=288.675


In [8]:
torch.save({
    'node_features': node_features,
    'window_labels': window_labels,
    "edge_features": edge_features,
    'window_scenarios': window_scenarios,
    "edge_index": graph_data['edge_index'].long().t().contiguous(),
},
    "data/processed_windowed_data_W24_S1_STRIDE18.pt"           
)

## Double Windowing

In [83]:
# Temporal parameters
WINDOW_SIZE = 256
DOUBLE_WINDOW_SIZE = 4
SUBSAMPLE_FACTOR = 1  # Reduced subsampling for more windows
STRIDE = 128
num_nodes = graph_data['node_features'].shape[1]

def preprocess_data(features, window_size, stride, subsample):
    """Process features into [windows, nodes] formatn"""
    # Slice data into overlapping windows
    features = features.unfold(0, window_size, stride).float()
    
    mean_values = features.mean(dim=2)  # Mean pooling
    max_values = features.max(dim=2).values  # Max pooling
    min_values = features.min(dim=2).values  # Min pooling
    std_values = features.std(dim=2)  # Standard deviation pooling
    
    features = torch.stack((mean_values, max_values, min_values, std_values), dim=1)
    features = einops.rearrange(features, "l f n -> l (f n)")
    
    # Subsample the windows
    return features[::subsample, :]


In [84]:
# Process features and labels
node_features = preprocess_data(graph_data['node_features'], WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
edge_features = preprocess_data(graph_data['edge_attr'], WINDOW_SIZE, STRIDE, SUBSAMPLE_FACTOR)
window_labels = graph_data['y'].unfold(0, WINDOW_SIZE, STRIDE).max(dim=2).values[::SUBSAMPLE_FACTOR, :]
window_scenarios = graph_data['scenario'].unfold(0, WINDOW_SIZE, STRIDE).mode(dim=2).values[::SUBSAMPLE_FACTOR, :]
print(node_features)
print(edge_features)
print(window_labels)
print(window_scenarios)

tensor[136874, 128] n=17519872 (67Mb) x∈[0., 69.985] μ=46.331 σ=27.324
tensor[136874, 272] n=37229728 (0.1Gb) x∈[-3.726e+03, 1.344e+04] μ=691.170 σ=1.419e+03
tensor[136874, 32] i64 n=4379968 (33Mb) x∈[0, 1] μ=0.008 σ=0.090
tensor[136874, 1] i32 0.5Mb x∈[1, 1000] μ=500.503 σ=288.676


In [85]:
def sliding_window(tensor, window_size, n):
    """
    Apply a sliding window operation on the input tensor.

    Args:
        tensor (torch.Tensor): Input tensor of shape [N, F].
        window_size (int): The size of the sliding window.

    Returns:
        torch.Tensor: Output tensor of shape [N, window_size * F].
    """
    N, F = tensor.shape
    # Create a padded tensor to handle the boundary conditions
    padded_tensor = torch.zeros((N + window_size - 1, F), dtype=tensor.dtype, device=tensor.device)
    padded_tensor[window_size - 1:] = tensor  # Place the original tensor at the end of the padded tensor

    # Initialize the output tensor
    output_tensor = torch.zeros((N, window_size * F), dtype=tensor.dtype, device=tensor.device)

    # Fill the output tensor with the sliding window values
    for i in range(N):
        # Get the window for the current time step
        window = padded_tensor[i:i + window_size, :]
        # Flatten the window and store it in the output tensor
        output_tensor[i, :] = window.flatten()

    return output_tensor

In [86]:
node_features = sliding_window(node_features, DOUBLE_WINDOW_SIZE, n=num_nodes)
edge_features = sliding_window(edge_features, DOUBLE_WINDOW_SIZE, n=num_edges)
print(node_features)
print(edge_features)

tensor[136874, 512] n=70079488 (0.3Gb) x∈[0., 69.985] μ=46.330 σ=27.324
tensor[136874, 1088] n=148918912 (0.6Gb) x∈[-3.726e+03, 1.344e+04] μ=691.164 σ=1.419e+03


In [87]:
torch.save({
    'node_features': node_features,
    'window_labels': window_labels,
    "edge_features": edge_features,
    'window_scenarios': window_scenarios,
    "edge_index": graph_data['edge_index'].long().t().contiguous(),
},
    f"data/processed_doublewindowed_data_W{WINDOW_SIZE}W{DOUBLE_WINDOW_SIZE}_S{SUBSAMPLE_FACTOR}_STRIDE{STRIDE}.pt"           
)